In [1]:
import numpy as np
import random

import h5py.highlevel
import astropy.io.fits as fits
from astropy.table import Table

import pandas as pd
import seaborn as sns
sns.set(style="ticks", context="talk", font_scale=1.3)

import os
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:4: H5pyDeprecationWarning: The h5py.highlevel module is deprecated, code should import directly from h5py, e.g. 'from h5py import File'.
  after removing the cwd from sys.path.


### We are going to determine the common wavelength range

In [ ]:
# find the range of data
lamLowerMin, lamLowerMax, lamUpperMin, lamUpperMax = 10000,  0,  10000,  0
for filename in filenames:
    name = os.path.splitext(filename)[0]
    data = _getData('./'+folder+'/'+filename, lower=0, upper=10000)
    lamLower = data['loglam'].head(1).item()
    lamUpper = data['loglam'].tail(1).item()
    if (lamLowerMin > lamLower):
        lamLowerMin = lamLower
        lamLowerMinName = name
    if (lamLowerMax < lamLower): 
        lamLowerMax = lamLower
        lamLowerMaxName = name
    if (lamUpperMin > lamUpper):
        lamUpperMin = lamUpper
        lamUpperMinName = name
    if (lamUpperMax < lamUpper): 
        lamUpperMax = lamUpper
        lamUpperMaxName = name
        
# for Serfert 1.9
# lamLowerMin, lamLowerMax, lamUpperMin, lamUpperMax
# 3.5778000354766846, 3.58270001411438, 3.870300054550171, 3.966200113296509

# for Serfert 2
# 3.578900098800659, 3.58270001411438, 3.962899923324585, 3.9646999835968018

###  We select from 4000A - 9000A, corresponding to

In [ ]:
lower = 4000
upper = 9000

### 获取数据，并且写入文件
举例： _getData('./Sy19/spec-1380-53084-0598.fits', lower=lower, upper=upper)

In [2]:
## get data
def _getData(filename, lower, upper, redshifted=False):
    obj=fits.open(filename)    
    data = Table.read(obj, hdu=1).to_pandas()
    z = Table.read(obj,hdu=2)['Z'].item()
    data['lam'] = np.power(10,data['loglam'])
    if redshifted:
        data['lam'] = data['lam']/(1+z)
    data = data[(data['lam']>lower) & (data['lam']<upper)]
    lam = np.linspace(lower,upper,upper-lower+1)
    flux = np.interp(lam, data['lam'], data['flux'])
    
    # normalize data
    std = np.std(flux)
    avg = np.mean(flux)
    eps = 0.001
    fluxNormalized = (flux - avg)/std
    fluxNormalized -= np.min(fluxNormalized)
    fluxNormalized += eps
    fluxNormalized /= np.max(fluxNormalized)
    dataSelected = pd.DataFrame({'lam': lam, 'flux': flux, 'fluxNormalized': fluxNormalized})
    _writefile(filename,dataSelected,z,lower, upper)
    return dataSelected, z

def _writefile(filename,data,z,lower, upper):
    name = os.path.splitext(filename)[0]
    outputName = name+'-'+str(int(lower))+'-'+str(int(upper))+'_raw.csv'
    f = open(outputName,'w')
    f.write('# Original file: '+str(filename)+'\n')
    f.write('# wavelength range from: '+str(lower)+' to '+str(upper)+'\n')
    f.write('# redshift: '+str(z) +'\n')
    data.to_csv(f)
    f.close()

### produce redshift corrected data from 4000A to 7500A

In [11]:
folder = 'Sy20_train'
filenames = [x for x in os.listdir(folder) if x.endswith(".fits")]
pixels = []
zs = []

for filename in filenames:
    data, z = _getData('./'+folder+'/'+filename, lower=3800, upper=7500, redshifted=True)
    pixels.append(data['fluxNormalized'].tolist())
    zs.append(z)

pixels = np.asarray(pixels).flatten()
zs = np.asarray(zs).flatten()
pixels.tofile('sy20-pixels-3800-7500-redshifted-normalized.csv', sep=",")
zs.tofile('sy20-zs-3800-7500-redshifted-normalized.csv', sep=",")

MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]
MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]
MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]
MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]
MC1TEMDN=-0.00000000000000E+00 / sp1 mech median temp                            [astropy.io.fits.card]
MC1TEMDN=-0.00000000000000E+00 / sp1 mech median temp                            [astropy.io.fits.card]


In [6]:
folder = 'Sy19_train'
filenames = [x for x in os.listdir(folder) if x.endswith(".fits")]
pixels = []
zs = []

for filename in filenames:
    data, z = _getData('./'+folder+'/'+filename, lower=3800, upper=7500, redshifted=True)
    pixels.append(data['fluxNormalized'].tolist())
    zs.append(z)

pixels = np.asarray(pixels).flatten()
zs = np.asarray(zs).flatten()
pixels.tofile('sy19-pixels-3800-7500-redshifted-normalized.csv', sep=",")
zs.tofile('sy19-zs-3800-7500-redshifted-normalized.csv', sep=",")

In [4]:
folder = 'Sy20_test'
filenames = [x for x in os.listdir(folder) if x.endswith(".fits")]

In [5]:
len(filenames)

53494

In [10]:
folder = 'Sy20_test'
filenames = [x for x in os.listdir(folder) if x.endswith(".fits")]
pixels = []
zs = []
i = 0
j = 0
for filename in filenames:
    data, z = _getData('./'+folder+'/'+filename, lower=3800, upper=7500, redshifted=True)
    pixels.append(data['fluxNormalized'].tolist())
    zs.append(z)
    i=i+1
    if (i==1000):
        print('--------------------',j,'----------------')
        pixels = np.asarray(pixels).flatten()
        zs = np.asarray(zs).flatten()
        pixels_filename = 'sy2-test-pixels-3800-7500-redshifted-normalized.'+str(j)+'.csv'
        pixels.tofile(pixels_filename, sep=",")
        zs_filename = 'sy2-test-zs-3800-7500-redshifted-normalized.'+str(j)+'.csv'
        zs.tofile(zs_filename, sep=",")
        pixels = []
        zs = []
        i=0
        j=j+1
pixels = np.asarray(pixels).flatten()
zs = np.asarray(zs).flatten()
pixels_filename = 'sy2-test-pixels-3800-7500-redshifted-normalized.'+str(j)+'.csv'
pixels.tofile(pixels_filename, sep=",")
zs_filename = 'sy2-test-zs-3800-7500-redshifted-normalized.'+str(j)+'.csv'

MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]
MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]


-------------------- 0 ----------------


MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1TRCB =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Bot                       [astropy.io.fits.card]
MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1TRCB =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Bot                       [astropy.io.fits.card]
MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]
MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]
MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]
MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]
MC1THT  =-0.00000000000000E+00 / sp1 mech Temp_Hartmann_Top                      [astropy.io.fits.card]
MC1THT  =-0.00000000000000E+00 / sp1 mech Temp_Hartmann_Top     

-------------------- 1 ----------------


MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1TRCB =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Bot                       [astropy.io.fits.card]
MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1TRCB =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Bot                       [astropy.io.fits.card]
MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1TRCB =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Bot                       [astropy.io.fits.card]
MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1TRCB =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Bot                       [astropy.io.fits.card]
MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]
MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top      

...

MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1TRCB =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Bot                       [astropy.io.fits.card]
MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1TRCB =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Bot                       [astropy.io.fits.card]


-------------------- 51 ----------------


MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1TRCB =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Bot                       [astropy.io.fits.card]
MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1TRCB =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Bot                       [astropy.io.fits.card]
MC1TEMDN=-0.00000000000000E+00 / sp1 mech median temp                            [astropy.io.fits.card]
MC1TEMDN=-0.00000000000000E+00 / sp1 mech median temp                            [astropy.io.fits.card]
MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1TRCB =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Bot                       [astropy.io.fits.card]
MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1TRCB =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Bot      

-------------------- 52 ----------------


MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]
MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]


In [8]:
folder = 'Sy19_test'
filenames = [x for x in os.listdir(folder) if x.endswith(".fits")]
pixels = []
zs = []

for filename in filenames:
    data, z = _getData('./'+folder+'/'+filename, lower=3800, upper=7500, redshifted=True)
    pixels.append(data['fluxNormalized'].tolist())
    zs.append(z)

pixels = np.asarray(pixels).flatten()
zs = np.asarray(zs).flatten()
pixels.tofile('sy19-test-pixels-3800-7500-redshifted-normalized.csv', sep=",")
zs.tofile('sy19-test-zs-3800-7500-redshifted-normalized.csv', sep=",")